In [136]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import wbgapi as wb

# 1. Clean the Given data

In [137]:
# Read the original data to python
data = pd.read_csv('homework_group_1-tidy.csv')

In [138]:
# If the country column is China.Hong.Kong,then change to Hong kong
## the match of hongkong is not correct, change the name to Hong Kong
data['country'] = np.where(data['country'] == 'China.Hong.Kong', 'Hong Kong',data['country'])

In [139]:
# Get the list of countries and use WBG matcher to match the country codes
countries = list(data['country'].unique())
country_code = wb.economy.coder(countries)
country_code = pd.DataFrame(country_code,index=[0])

In [140]:
# convert the country_code from row to column
country_code = country_code.transpose()
country_code = country_code.reset_index()
country_code = country_code.rename(columns={'index': 'country',0:'country code'})

In [141]:
# merge back to the original data
data = pd.merge(data,country_code,on='country')

In [142]:
# Seperate the year from the date
data['year'] = data['date'].apply(lambda x: x.split('-')[0])

# 2. Get and clean the GDP data

In [143]:
# search for the gpd per capita data
wb.search("gdp per capita")

ID,Name,Field,Value
NY.GDP.PCAP.CD,GDP per capita (current US$),IndicatorName,GDP per capita (current US$)
NY.GDP.PCAP.CD,GDP per capita (current US$),Longdefinition,...GDP per capita is gross domestic product divided by midyear population. GDP is the sum of...
NY.GDP.PCAP.CN,GDP per capita (current LCU),IndicatorName,GDP per capita (current LCU)
NY.GDP.PCAP.CN,GDP per capita (current LCU),Longdefinition,...GDP per capita is gross domestic product divided by midyear population. GDP is the sum of...
NY.GDP.PCAP.KD,GDP per capita (constant 2015 US$),IndicatorName,GDP per capita (constant 2015 US$)
NY.GDP.PCAP.KD,GDP per capita (constant 2015 US$),Longdefinition,...GDP per capita is gross domestic product divided by midyear population. GDP is the sum of...
NY.GDP.PCAP.KD.ZG,GDP per capita growth (annual %),IndicatorName,GDP per capita growth (annual %)
NY.GDP.PCAP.KD.ZG,GDP per capita growth (annual %),Longdefinition,...Annual percentage growth rate of GDP per capita based on constant local currency. GDP per capita is gross domestic product...
NY.GDP.PCAP.KN,GDP per capita (constant LCU),IndicatorName,GDP per capita (constant LCU)
NY.GDP.PCAP.KN,GDP per capita (constant LCU),Longdefinition,...GDP per capita is gross domestic product divided by midyear population. GDP at purchaser's...


In [185]:
# Get gdp per capita data using WBG API
GDP = wb.data.DataFrame({"NY.GDP.MKTP.PP.CD", "NY.GDP.PCAP.PP.CD"}, list(data['country code'].unique()))

In [186]:
GDP = GDP.reset_index()

In [187]:
# Change the dataframe format and rename
GDP = pd.melt(GDP, id_vars = ['economy','series'],value_vars=list(GDP.columns))
GDP = GDP.rename(columns={'economy': 'country code', 'variable':'year'})

In [188]:
# Rename the values in variable column, delete YR
GDP['year'] = GDP['year'].str[2:]

In [189]:
# Seprate GDP and GDP per capita data
GDP_capita = GDP[GDP['series'] == 'NY.GDP.PCAP.PP.CD'].drop('series', axis=1).rename(columns={'value': 'GDP per capita'})
GDP = GDP[GDP['series'] == 'NY.GDP.MKTP.PP.CD'].drop('series', axis=1).rename(columns={'value': 'GDP'})

In [190]:
# merge the gdp data back to the original data
data = pd.merge(data,GDP,on=['country code','year'], how='left')
data = pd.merge(data,GDP_capita, on=['country code','year'],how = 'left')

In [116]:
GDP

,country code,year,GDP per Capita
0,AUS,1960,NaN
1,CAN,1960,NaN
2,CHE,1960,NaN
3,DNK,1960,NaN
4,GBR,1960,NaN
...,...,...,...
1173,SAU,2021,48710.947680
1174,SGP,2021,116486.473211
1175,SWE,2021,59323.964612
1176,THA,2021,18760.755222


In [127]:
GDP['year'] = GDP['year'].astype(int)

In [133]:
GDP2 = GDP[GDP['year']>=2000]

In [134]:
GDP2

,country code,year,GDP per Capita
760,AUS,2000,26532.564360
761,CAN,2000,29362.084161
762,CHE,2000,36772.694618
763,DNK,2000,28658.124955
764,GBR,2000,26472.193714
...,...,...,...
1173,SAU,2021,48710.947680
1174,SGP,2021,116486.473211
1175,SWE,2021,59323.964612
1176,THA,2021,18760.755222


In [123]:
country_code

,country,country code
0,Australia,AUS
1,Canada,CAN
2,Hong Kong,HKG
3,Denmark,DNK
4,Hungary,HUN
5,India,IND
6,Japan,JPN
7,Kuwait,KWT
8,Mexico,MEX
9,New.Zealand,NZL


In [126]:
GDP2 = wb.data.DataFrame({"NY.GDP.MKTP.PP.CD", "NY.GDP.PCAP.PP.CD"}, list(data['country code'].unique()))

In [128]:
GDP2

YR1960  YR1961  YR1962  YR1963  YR1964  YR1965  \
economy series                                                              
AUS     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
CAN     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
CHE     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
DNK     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
GBR     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
HKG     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
HUN     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
IND     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
JPN     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
KWT     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
MEX     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
NOR     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
NZL     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
PHL     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
SAU     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
SGP     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
SWE     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
THA     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
ZAF     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   

                           YR1966  YR1967  YR1968  YR1969  ...        YR2012  \
economy series                                             ...                 
AUS     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN  ...  9.740675e+11   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN  ...  4.284730e+04   
CAN     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN  ...  1.468098e+12   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN  ...  4.229097e+04   
CHE     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN  ...  4.792807e+11   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN  ...  5.993360e+04   
DNK     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN  ...  2.505246e+11   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN  ...  4.480397e+04   
GBR     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN  ...  2.453352e+12   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN  ...  3.

In [129]:
data

,Unnamed: 0,country,date,changeFX,diffInt,country code,year,GDP per Capita
0,1,Australia,2000-01-31,0.025694,-0.000517,AUS,2000,26532.56436
1,2,Australia,2000-02-29,0.037704,-0.000326,AUS,2000,26532.56436
2,3,Australia,2000-03-31,0.011792,-0.000346,AUS,2000,26532.56436
3,4,Australia,2000-04-28,0.039484,-0.000223,AUS,2000,26532.56436
4,5,Australia,2000-05-31,0.022531,-0.000508,AUS,2000,26532.56436
...,...,...,...,...,...,...,...,...
2635,2636,United.Kingdom,2010-08-31,0.018821,0.000215,GBR,2010,36576.58654
2636,2637,United.Kingdom,2010-09-30,-0.024996,0.000222,GBR,2010,36576.58654
2637,2638,United.Kingdom,2010-10-29,-0.014490,0.000219,GBR,2010,36576.58654
2638,2639,United.Kingdom,2010-11-30,0.026268,0.000205,GBR,2010,36576.58654


In [132]:
GDP2

YR1960  YR1961  YR1962  YR1963  YR1964  YR1965  \
economy series                                                              
AUS     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
CAN     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
CHE     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
DNK     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
GBR     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
HKG     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
HUN     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
IND     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
JPN     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
KWT     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
MEX     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
NOR     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
NZL     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
PHL     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
SAU     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
SGP     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
SWE     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
THA     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
ZAF     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN     NaN     NaN   

                           YR1966  YR1967  YR1968  YR1969  ...        YR2012  \
economy series                                             ...                 
AUS     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN  ...  9.740675e+11   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN  ...  4.284730e+04   
CAN     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN  ...  1.468098e+12   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN  ...  4.229097e+04   
CHE     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN  ...  4.792807e+11   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN  ...  5.993360e+04   
DNK     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN  ...  2.505246e+11   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN  ...  4.480397e+04   
GBR     NY.GDP.MKTP.PP.CD     NaN     NaN     NaN     NaN  ...  2.453352e+12   
        NY.GDP.PCAP.PP.CD     NaN     NaN     NaN     NaN  ...  3.